In [29]:
import numpy as np
import pandas as pd
import sklearn
import scipy
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
#from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import preprocessing
data = pd.read_csv(r'CDS_vectorization_v1.csv',dtype={'Error_Label':object} )
print(data)
#data.head()

          ID       v1       v2       v3       v4       v5       v6       v7  \
0      10001  0.41463  1.18500  1.12590 -0.63784 -1.60940 -0.19869 -0.23010   
1      10002 -1.68950 -0.63024 -0.23417 -2.21810 -0.36157 -0.52164 -0.31883   
2      10003 -1.71670  2.40120  0.44570  1.14400 -0.53560 -0.69040  0.39990   
3      10004 -0.07260  0.31790 -1.38690  1.83000 -1.08930 -1.15900  0.05750   
4      10005  0.32088 -1.08450 -1.15780 -0.63845  1.03060  0.38707  0.95113   
...      ...      ...      ...      ...      ...      ...      ...      ...   
62178  96207 -1.45150 -0.95600 -0.89920 -0.80340 -0.83730  2.07520 -2.71140   
62179  96208  1.27570 -1.32360  2.00620 -1.61870 -0.72743  0.58177 -2.55890   
62180  96209  0.35809 -2.22570  0.22366 -2.89240  0.43210 -1.64220 -0.12659   
62181  96210  0.40804 -2.27950 -0.83962 -1.03440  1.04950  1.32130  1.27670   
62182  96211 -0.74982  0.35505 -0.47464 -2.38330 -1.45550 -0.62356 -1.79640   

            v8       v9  ...      v491     v492    

In [2]:
# split labels and return new dataframe
#Create independent and Dependent Features
#data=split_label(data)
def split_label_dict(df):
    # Change 'label' to whatever naming used in the original DataFrame.
    temp_df = df['Error_Label'].str.split("", n = -1, expand = True)
    for i in range(1, 19):
        df[f'l{6}'] = temp_df[6]
        df[f'l{6}'] = pd.to_numeric(df[f'l{6}'])
    return df

newdata=split_label_dict(data)
#print(newdata)
#Create independent and Dependent Features
columns = newdata.columns.tolist()
# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["l6"]]
# Store the variable we are predicting 
target = "l6"
# Define a random state 
state = np.random.RandomState(69)
X = newdata[columns]
Y = newdata[target]
# Print the shapes of X & Y
print(X.shape)
print(Y.shape)

(62183, 501)
(62183,)


In [30]:
# split labels and return new dataframe
#Create independent and Dependent Features
#data=split_label(data)
def split_label_dict(df):
    # Change 'label' to whatever naming used in the original DataFrame.
    temp_df = df['Error_Label'].str.split("", n = -1, expand = True)
    for i in range(1, 19):
        df[f'l{6}'] = temp_df[6]
        df[f'l{6}'] = pd.to_numeric(df[f'l{6}'])
    return df

newdata=split_label_dict(data)
#print(newdata)
#Create independent and Dependent Features
df = newdata.values[:, 1:500]
X = pd.DataFrame(df) 
state = np.random.RandomState(69)
Y = newdata["l6"]
print(X)
print(Y)

           0        1        2        3        4        5        6        7    \
0      0.41463    1.185   1.1259 -0.63784  -1.6094 -0.19869  -0.2301  -0.6919   
1      -1.6895 -0.63024 -0.23417  -2.2181 -0.36157 -0.52164 -0.31883   1.0674   
2      -1.7167   2.4012   0.4457    1.144  -0.5356  -0.6904   0.3999   1.6333   
3      -0.0726   0.3179  -1.3869     1.83  -1.0893   -1.159   0.0575   0.7245   
4      0.32088  -1.0845  -1.1578 -0.63845   1.0306  0.38707  0.95113  0.30132   
...        ...      ...      ...      ...      ...      ...      ...      ...   
62178  -1.4515   -0.956  -0.8992  -0.8034  -0.8373   2.0752  -2.7114   1.6391   
62179   1.2757  -1.3236   2.0062  -1.6187 -0.72743  0.58177  -2.5589  0.46077   
62180  0.35809  -2.2257  0.22366  -2.8924   0.4321  -1.6422 -0.12659 -0.65374   
62181  0.40804  -2.2795 -0.83962  -1.0344   1.0495   1.3213   1.2767  0.71817   
62182 -0.74982  0.35505 -0.47464  -2.3833  -1.4555 -0.62356  -1.7964  -2.6144   

           8        9    ..

In [31]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X, Y)
print(X_resampled.shape, y_resampled.shape )
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y)))
print('Resampled dataset shape {}'.format(Counter(y_resampled)))

(58900, 499) (58900,)
Original dataset shape Counter({0: 32733, 1: 29450})
Resampled dataset shape Counter({0: 29450, 1: 29450})


In [32]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X_resampled,y_resampled,test_size=0.3,random_state=69)

In [33]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [34]:
# Fitting the classifier into the Training set

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 500)
classifier.fit(X_train,y_train)
# Predicting the test set results
y_predict = classifier.predict(X_test)
print(accuracy_score(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.5276740237691002


col_0,0,1
l6,,
0,4036,4785
1,3561,5288


In [28]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pickle
results=classification_report(y_test,y_predict)
macro=f1_score(y_test, y_predict, average='macro')
micro=f1_score(y_test, y_predict, average='micro')
weighted=f1_score(y_test, y_predict, average='weighted')
avg=f1_score(y_test, y_predict)
#accuracy_score=accuracy_score(y_test,y_predict)
cm = confusion_matrix(y_test, y_predict)
roc=roc_auc_score(y_test,y_predict)
print(results)
print(macro)
print(micro)
print(roc)
print(cm)

              precision    recall  f1-score   support

           0       0.52      0.55      0.53      8821
           1       0.52      0.49      0.50      8849

    accuracy                           0.52     17670
   macro avg       0.52      0.52      0.52     17670
weighted avg       0.52      0.52      0.52     17670

0.5165661515024056
0.5170345217883419
0.5170851429664329
[[4843 3978]
 [4556 4293]]


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
# Predicting the test set results
Y_Pred = classifier.predict(X_test)
# Making the Confusion Matrix 
print(accuracy_score(y_test,Y_Pred))
cm = confusion_matrix(y_test, Y_Pred)
print(cm)
print(classification_report(y_test,Y_Pred))

In [5]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek()
X_res,y_res=smk.fit_sample(X_train.astype('float'),y_train)
print(X_res.shape,y_res.shape)
#print(X_res)
#print(y_res)

(71156, 501) (71156,)


In [6]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({0: 35578, 1: 7950})
Resampled dataset shape Counter({0: 35578, 1: 35578})


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import pickle
# Fitting the classifier into the Training set
classifier = RandomForestClassifier(n_estimators = 300, random_state = 200)
model=classifier.fit(X_res,y_res)

# Save History as Pickle

filename = 'models_results/l16RFM_tomek.pickle'
with open(filename, 'wb') as fh:
    pickle.dump(model, fh)





In [8]:
# Predicting the test set results
Y_Pred = classifier.predict(X_test)

# Making the Confusion Matrix 

print(accuracy_score(y_test,Y_Pred))
cm = confusion_matrix(y_test, Y_Pred)
print(cm)


0.8076655052264808
[[14889   371]
 [ 3217   178]]


In [9]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score 
import pickle
results=classification_report(y_test,Y_Pred)
#accuracy_score=accuracy_score(y_test,Y_Pred)
cm = confusion_matrix(y_test, Y_Pred)
print(cm)
print(results) 
#print(accuracy_score)

# Save Results as Pickle
filename = 'models_results/l16RFtomek.pickle'
with open(filename, 'wb') as fh:
    pickle.dump([results, cm], fh)

[[14889   371]
 [ 3217   178]]
              precision    recall  f1-score   support

           0       0.82      0.98      0.89     15260
           1       0.32      0.05      0.09      3395

    accuracy                           0.81     18655
   macro avg       0.57      0.51      0.49     18655
weighted avg       0.73      0.81      0.75     18655



In [9]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

# take the second column because the classifier outputs scores for
# the 0 class as well
preds = Y_Pred[:,1]

# fpr means false-positive-rate
# tpr means true-positive-rate
fpr, tpr, _ = metrics.roc_curve(y_test, preds)

auc_score = metrics.auc(fpr, tpr)

# clear current figure
plt.clf()

plt.title('ROC Curve')
plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_score))

# it's helpful to add a diagonal to indicate where chance 
# scores lie (i.e. just flipping a coin)
plt.plot([0,1],[0,1],'r--')

plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.legend(loc='lower right')
plt.show()

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed